In [81]:
from keras.models import Sequential
import numpy as numpy 

In [168]:
from numpy import genfromtxt
my_data = genfromtxt('mfcc_vectors.csv', delimiter=',')


In [169]:
# print(my_data)
print(my_data.shape)

(2000, 21)


In [170]:
Y = numpy.copy(my_data[:, 20:])
print(Y.shape)

(2000, 1)


In [171]:
X = numpy.copy(my_data[:, :12])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(2000, 12)
(1, 12)
(1, 12)
(2000, 12)


In [214]:
from keras import utils as np_utils

one_hot_labels = np_utils.to_categorical(Y, num_classes=6)
print(one_hot_labels)

[[ 0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]]


In [226]:
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# model = Sequential()
# model.add(Dense(32, activation='relu', input_dim=12))
# model.add(Dense(10, activation='softmax'))

# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)


# MultiLayer Perceptron
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(32, activation='relu', input_dim=12))
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)
# score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/10
2000/2000 [==============================] - 0s 36us/step - loss: 1.6928 - acc: 0.1935
Epoch 2/10
2000/2000 [==============================] - 0s 38us/step - loss: 1.5421 - acc: 0.2845
Epoch 3/10
2000/2000 [==============================] - 0s 38us/step - loss: 1.4868 - acc: 0.3315
Epoch 4/10
2000/2000 [==============================] - 0s 39us/step - loss: 1.4266 - acc: 0.3515
Epoch 5/10
2000/2000 [==============================] - 0s 40us/step - loss: 1.3747 - acc: 0.3815
Epoch 6/10
2000/2000 [==============================] - 0s 41us/step - loss: 1.3391 - acc: 0.3835
Epoch 7/10
2000/2000 [==============================] - 0s 41us/step - loss: 1.3094 - acc: 0.4120
Epoch 8/10
2000/2000 [==============================] - 0s 38us/step - loss: 1.2831 - acc: 0.4190
Epoch 9/10
2000/2000 [==============================] - 0s 40us/step - loss: 1.2556 - acc: 0.4425
Epoch 10/10
2000/2000 [==============================] - 0s 37us/step - loss: 1.2368 - acc: 0.4580


In [227]:
test_model = genfromtxt('test_vectors.csv', delimiter=',')
print(test_model.shape)

test_X = numpy.copy(test_model[:, :12])
print(test_X.shape)

normalized_test_X = (test_X - mean) / std_deviation
print(normalized_test_X.shape)

test_Y = numpy.copy(test_model[:, 20:])
print(test_Y.shape)
test_labels = np_utils.to_categorical(test_Y, num_classes=6)


(2200, 21)
(2200, 12)
(2200, 12)
(2200, 1)


In [230]:
print(model.test_on_batch(test_X, test_labels, sample_weight=None))
print(model.metrics_names)
predictions = model.predict(test_X)

[array(4.3494133949279785, dtype=float32), array(0.46409091353416443, dtype=float32)]
['loss', 'acc']


In [255]:
b = [sum(predictions[current: current+40]) for current in range(0, len(predictions), 40)]
predicted_Y = []
for row in b:
    predicted_Y.append(row.argmax(axis=0))
    
print(predicted_Y)
print(test_Y[::40].T[0])

diff = predicted_Y - test_Y[::40].T[0]

print(len(predicted_Y))
print(sum(x == 0 for x in diff))

[3, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 1, 3, 3, 1, 4, 4, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 4, 1, 3, 4, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3]
[ 2.  1.  4.  1.  4.  1.  5.  5.  1.  1.  3.  3.  5.  3.  5.  2.  4.  5.
  2.  4.  3.  1.  1.  4.  1.  5.  1.  4.  4.  1.  1.  5.  3.  1.  1.  3.
  5.  5.  4.  4.  1.  5.  4.  1.  3.  3.  2.  3.  3.  1.  1.  5.  3.  3.
  5.]
55
26
